# Ensuring Consistency

**Activity Overview**: Ensure consistency by identifying and resolving conflicting values across datasets.

## Title: Customer Address Discrepancies

**Task**: Address customer address mismatches between CRM and marketing databases.

**Steps**:
1. Compare customer addresses in the CRM with those in the marketing database.
2. Identify records with conflicting address information.
3. Propose a method to consolidate records with verified addresses.

In [ ]:
# Write your code from here
import pandas as pd

def find_address_discrepancies(crm_data_path, marketing_data_path):
    """
    Identifies customer address discrepancies between CRM and marketing databases.

    Args:
        crm_data_path (str): Path to the CSV file containing CRM customer data.
        marketing_data_path (str): Path to the CSV file containing marketing customer data.

    Returns:
        pandas.DataFrame: A DataFrame containing records with conflicting address information,
                          or None if an error occurs.
    """
    try:
        # Load the datasets
        crm_df = pd.read_csv(crm_data_path)
        marketing_df = pd.read_csv(marketing_data_path)

        # Check for required columns
        if not all(col in crm_df.columns for col in ['Customer_ID', 'Address']):
            print("Error: CRM data must contain 'Customer_ID' and 'Address' columns.")
            return None
        if not all(col in marketing_df.columns for col in ['Customer_ID', 'Address']):
            print("Error: Marketing data must contain 'Customer_ID' and 'Address' columns.")
            return None

        # Merge the datasets on Customer_ID
        merged_df = pd.merge(crm_df, marketing_df, on='Customer_ID', suffixes=('_CRM', '_Marketing'))

        # Identify discrepancies
        discrepancy_df = merged_df[merged_df['Address_CRM'] != merged_df['Address_Marketing']].copy()

        if discrepancy_df.empty:
            print("No address discrepancies found.")
            return pd.DataFrame()  # Return an empty DataFrame

        return discrepancy_df[['Customer_ID', 'Address_CRM', 'Address_Marketing']]

    except FileNotFoundError as e:
        print(f"Error: File not found: {e.filename}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def consolidate_addresses(discrepancy_df, verified_addresses_path=None):
    """
    Proposes a method to consolidate records with verified addresses.

    Args:
        discrepancy_df (pandas.DataFrame): DataFrame containing address discrepancies
                                           (output from find_address_discrepancies).
        verified_addresses_path (str, optional): Path to a CSV file containing verified addresses.
                                                 If None, user input is used. Defaults to None.

    Returns:
        pandas.DataFrame: A DataFrame with consolidated addresses, or None if input is invalid.
    """
    if discrepancy_df is None or discrepancy_df.empty:
        print("No discrepancies to consolidate.")
        return None

    if 'Customer_ID' not in discrepancy_df.columns:
        print("Error: Input DataFrame must contain 'Customer_ID' column.")
        return None

    consolidated_addresses = {}
    if verified_addresses_path:
        try:
            verified_df = pd.read_csv(verified_addresses_path)
            if not all(col in verified_df.columns for col in ['Customer_ID', 'Verified_Address']):
                print("Error: Verified addresses file must contain 'Customer_ID' and 'Verified_Address' columns.")
                verified_addresses = {} # set to empty dict to avoid errors
            else:
                verified_addresses = pd.Series(verified_df.Verified_Address.values, index=verified_df.Customer_ID).to_dict()
        except FileNotFoundError:
            print(f"Warning: Verified addresses file not found.  Using user input.")
            verified_addresses = {}
    else:
        verified_addresses = {}

    for index, row in discrepancy_df.iterrows():
        customer_id = row['Customer_ID']
        print(f"\nDiscrepancy for Customer ID: {customer_id}")
        print(f"  CRM Address:     {row['Address_CRM']}")
        print(f"  Marketing Address: {row['Address_Marketing']}")

        if customer_id in verified_addresses:
            verified_address = verified_addresses[customer_id]
            print(f"  Verified Address: {verified_address} (from file)")
            consolidated_addresses[customer_id] = verified_address
        else:
            verified_address = input("  Enter the verified address (or type 'skip' to skip): ")
            if verified_address.lower() != 'skip':
                consolidated_addresses[customer_id] = verified_address

    if not consolidated_addresses:
        print("No addresses were consolidated.")
        return pd.DataFrame()

    consolidated_df = pd.DataFrame(list(consolidated_addresses.items()), columns=['Customer_ID', 'Consolidated_Address'])
    return consolidated_df



if __name__ == '__main__':
    # Create sample data files
    crm_data = {'Customer_ID': [101, 102, 103, 104, 105],
                'Name': ['John Doe', 'Jane Smith', 'Peter Jones', 'Alice Brown', 'Bob Williams'],
                'Address': ['123 Main St', '456 Oak Ave', '789 Pine Ln', '101 Elm St', '222 Oak Ave']}
    marketing_data = {'Customer_ID': [101, 102, 103, 104, 105],
                      'Email': ['john.doe@example.com', 'jane.smith@example.com', 'peter.jones@sample.org', 'alice.brown@email.net', 'bob.williams@work.net'],
                      'Address': ['123 Main St', '456 Oak Ave', '789 Pine Lane', '101 Elm St', '333 Maple Dr']}
    crm_df = pd.DataFrame(crm_data)
    marketing_df = pd.DataFrame(marketing_data)
    crm_file_path = 'crm_data.csv'
    marketing_file_path = 'marketing_data.csv'
    crm_df.to_csv(crm_file_path, index=False)
    marketing_df.to_csv(marketing_file_path, index=False)

    # Create a sample verified addresses file
    verified_addresses_data = {'Customer_ID': [103, 105],
                               'Verified_Address': ['789 Pine Ln', '222 Oak Ave']}
    verified_addresses_df = pd.DataFrame(verified_addresses_data)
    verified_addresses_file_path = 'verified_addresses.csv'
    verified_addresses_df.to_csv(verified_addresses_file_path, index=False)

    # Find discrepancies
    discrepancies_df = find_address_discrepancies(crm_file_path, marketing_file_path)

    if discrepancies_df is not None and not discrepancies_df.empty:
        print("Address Discrepancies:")
        print(discrepancies_df)

        # Consolidate addresses (example with verified addresses file)
        consolidated_df = consolidate_addresses(discrepancies_df, verified_addresses_file_path)
        if consolidated_df is not None and not consolidated_df.empty:
            print("\nConsolidated Addresses (using verified addresses file):")
            print(consolidated_df)

        # Consolidate addresses (example with user input)
        consolidated_df_user_input = consolidate_addresses(discrepancies_df)
        if consolidated_df_user_input is not None and not consolidated_df_user_input.empty:
            print("\nConsolidated Addresses (using user input):")
            print(consolidated_df_user_input)
    elif discrepancies_df is not None:
        print("No address discrepancies found.")
    else:
        print("An error occurred.")


Address Discrepancies:
   Customer_ID  Address_CRM Address_Marketing
2          103  789 Pine Ln     789 Pine Lane
4          105  222 Oak Ave      333 Maple Dr

Discrepancy for Customer ID: 103
  CRM Address:     789 Pine Ln
  Marketing Address: 789 Pine Lane
  Verified Address: 789 Pine Ln (from file)

Discrepancy for Customer ID: 105
  CRM Address:     222 Oak Ave
  Marketing Address: 333 Maple Dr
  Verified Address: 222 Oak Ave (from file)

Consolidated Addresses (using verified addresses file):
   Customer_ID Consolidated_Address
0          103          789 Pine Ln
1          105          222 Oak Ave

Discrepancy for Customer ID: 103
  CRM Address:     789 Pine Ln
  Marketing Address: 789 Pine Lane

Discrepancy for Customer ID: 105
  CRM Address:     222 Oak Ave
  Marketing Address: 333 Maple Dr
